In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/melting-point/sample_submission.csv
/kaggle/input/melting-point/train.csv
/kaggle/input/melting-point/test.csv


In [2]:
train= pd.read_csv('/kaggle/input/melting-point/train.csv')
train.head()

,id,SMILES,Tm,Group 1,Group 2,Group 3,Group 4,Group 5,Group 6,Group 7,...,Group 415,Group 416,Group 417,Group 418,Group 419,Group 420,Group 421,Group 422,Group 423,Group 424
0,2175,FC1=C(F)C(F)(F)C1(F)F,213.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1222,c1ccc2c(c1)ccc3Nc4ccccc4c23,407.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2994,CCN1C(C)=Nc2ccccc12,324.15,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1704,CC#CC(=O)O,351.15,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2526,CCCCC(S)C,126.15,2,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
train.info

<bound method DataFrame.info of         id                       SMILES      Tm  Group 1  Group 2  Group 3  \
0     2175        FC1=C(F)C(F)(F)C1(F)F  213.15        0        0        0   
1     1222  c1ccc2c(c1)ccc3Nc4ccccc4c23  407.15        0        0        0   
2     2994          CCN1C(C)=Nc2ccccc12  324.15        2        1        0   
3     1704                   CC#CC(=O)O  351.15        1        0        0   
4     2526                    CCCCC(S)C  126.15        2        3        0   
...    ...                          ...     ...      ...      ...      ...   
2657   707                       ClCCBr  256.45        0        1        0   
2658  1573               N#CC(Cl)(Cl)Cl  231.15        0        0        0   
2659  1634           Cc1ccc2c(C)cccc2c1  256.25        0        0        0   
2660  3022        CCC(=O)c1ccc2ccccc2c1  333.15        1        1        0   
2661  1673            Brc1ccc(cc1)N(C)C  328.15        2        0        0   

      Group 4  Group 5  Group 6

In [4]:
train.isnull().any().sum()

0

In [5]:
train_cat= train.select_dtypes(include= 'object').columns.to_list()
print(train_cat)

['SMILES']


In [6]:
x_train= train.drop(columns=['Tm'], axis=1)
y_train= train['Tm']

In [7]:
!pip install -U scikit-learn scikeras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 54.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.7.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.1 which is incompatible.


In [8]:
import sklearn
import scikeras
print("sklearn:", sklearn.__version__)
print("scikeras:", scikeras.__version__)

2025-09-08 15:39:59.780143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757346000.082732      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757346000.175275      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


sklearn: 1.7.1
scikeras: 0.13.0


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from scikeras.wrappers import KerasRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score

In [10]:
def build_model(n_features, l2= 1e-4, dropout= 0.15, lr= 1e-3):
    model= keras.Sequential([
        layers.Input(shape=(n_features,)),
        layers.Dense(30, activation='relu', kernel_regularizer= regularizers.l2(l2)),
        layers.Dropout(dropout),
        layers.Dense(30, activation='relu', kernel_regularizer= regularizers.l2(l2)),
        layers.Dense(1)
    ])

    model.compile(optimizer= keras.optimizers.Adam(learning_rate= lr), loss= 'mae', metrics=['mae'])
    return model

In [11]:
x_train= x_train.drop(columns=['id'], axis=1)

In [12]:
num_cols= x_train.select_dtypes(include= np.number).columns.to_list()
#print(num_cols)

In [13]:
cat_cols= x_train.select_dtypes(include='object').columns.to_list()
print(cat_cols)

categories = []
for c in cat_cols:
    vals = x_train[c].dropna().unique()
    categories.append(np.sort(vals)) 

['SMILES']


In [14]:
preprocessing= ColumnTransformer(transformers=[
    ('scaling', StandardScaler(), num_cols),
    ('ohe', OneHotEncoder(categories=categories, handle_unknown='ignore', sparse_output=False), cat_cols)
],
    remainder='passthrough'
)



In [15]:
X_train_transformed = preprocessing.fit_transform(x_train)   # fits ohe with pre-specified categories
n_features = X_train_transformed.shape[1]
print("n_features (transformed):", n_features)

n_features (transformed): 3086


In [16]:
nn= KerasRegressor(
    model=build_model, 
                    model__n_features= n_features,
                    epochs=200, batch_size=64, verbose=0,
                    callbacks=[
                        keras.callbacks.EarlyStopping(patience=20,
                                                      restore_best_weights=True),
                        keras.callbacks.ReduceLROnPlateau(patience=8, factor=0.5)
                    ]
)

In [17]:
pipeline= Pipeline([
    ('preprocessing', preprocessing),
    ('model', nn)
])

In [18]:
cv= KFold(n_splits=5, shuffle= True, random_state=42)
scores= -cross_val_score(pipeline, x_train, y_train, cv=cv, scoring="neg_mean_absolute_error")
print("CV MAE:", scores.mean(), "+/-", scores.std())

2025-09-08 15:40:17.590503: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae
  current = self.get_monitor_value(logs)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/callback_list.py:145: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae,learning_rate.
  callback.on_epoch_end(epoch, logs)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae
  current = self.get_monitor_value(logs)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/callback_list

CV MAE: 40.68187388122697 +/- 1.3033539717946165


In [19]:
pipeline.fit(x_train, y_train)

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae
  current = self.get_monitor_value(logs)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/callback_list.py:145: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae,learning_rate.
  callback.on_epoch_end(epoch, logs)


,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('scaling', ...), ('ohe', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [20]:
test=pd.read_csv('/kaggle/input/melting-point/test.csv')

In [21]:
test_id= test['id']
x_test = test.drop(columns=['id'])

In [22]:
y_pred = pipeline.predict(x_test)

In [23]:
submission = pd.DataFrame({
    'id': test_id,
    'Tm': y_pred
})
submission.to_csv("submission.csv", index=False)

print("Predictions saved to submission.csv")

Predictions saved to submission.csv


In [24]:
from sklearn.ensemble import RandomForestRegressor

In [25]:
pipeline2= Pipeline([
    ('preprocessing', preprocessing),
    ('model',RandomForestRegressor(n_estimators=500,       
    max_depth=None,         
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1))
])

In [26]:
pipeline2.fit(x_train, y_train)

,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('scaling', ...), ('ohe', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [27]:
y_2_pred= pipeline2.predict(x_test)

In [28]:
submission= pd.DataFrame({
    'id': test['id'],
    'Tm': y_2_pred
})

In [29]:
submission.to_csv('submission_RfR.csv', index= False)
print('file saved')

file saved


In [30]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

xgb_pipe= Pipeline([
    ('preprocessing', preprocessing),
    ('model',XGBRegressor(
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1))
])

In [31]:
param_dist = {
    'model__n_estimators': [200, 400, 600],        # number of trees
    'model__max_depth': [4, 6, 8, 10],             # depth of trees
    'model__learning_rate': [0.01, 0.05, 0.1],     # shrinkage
    'model__subsample': [0.7, 0.8, 1.0],           # row sampling
    'model__colsample_bytree': [0.7, 0.8, 1.0],    # feature sampling
    'model__min_child_weight': [1, 3, 5],          # min sum of instance weight
    'model__gamma': [0, 0.1, 0.2],                 # min loss reduction
    'model__reg_lambda': [1, 1.5, 2.0]             # L2 regularization
}

In [32]:
search = RandomizedSearchCV(
    estimator=xgb_pipe,
    param_distributions=param_dist,
    n_iter=30,                     # try 30 random combos
    cv=5,
    scoring="neg_mean_absolute_error",
    verbose=2,
    random_state=42,
    n_jobs=-1
)

search.fit(x_train, y_train)

print("Best MAE:", -search.best_score_)
print("Best params:", search.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] END model__colsample_bytree=1.0, model__gamma=0.1, model__learning_rate=0.05, model__max_depth=6, model__min_child_weight=5, model__n_estimators=200, model__reg_lambda=2.0, model__subsample=0.8; total time=   6.0s
[CV] END model__colsample_bytree=1.0, model__gamma=0.1, model__learning_rate=0.05, model__max_depth=6, model__min_child_weight=5, model__n_estimators=200, model__reg_lambda=2.0, model__subsample=0.8; total time=   5.3s
[CV] END model__colsample_bytree=0.7, model__gamma=0, model__learning_rate=0.1, model__max_depth=8, model__min_child_weight=3, model__n_estimators=600, model__reg_lambda=1.5, model__subsample=1.0; total time=  12.7s
[CV] END model__colsample_bytree=0.8, model__gamma=0.2, model__learning_rate=0.05, model__max_depth=8, model__min_child_weight=3, model__n_estimators=400, model__reg_lambda=2.0, model__subsample=1.0; total time=  10.0s
[CV] END model__colsample_bytree=0.8, model__gamma=0.2, model__le

In [33]:
y_pred = search.best_estimator_.predict(x_test)

submission = pd.DataFrame({
    'id': test['id'],
    'Tm': y_pred
})
submission.to_csv("submission_xgb.csv", index=False)